# Make Dataset

features

In [1]:
#different make sequence function
import numpy as np
import pandas as pd
import os

robot_dof = 7

# Function to generate sequence names
def generate_sequence(base_name, indices):
    return [f'{base_name}{i}' for i in indices]

# Automate the sequence generation using the function
indices = range(robot_dof)  # Since the indices go from 0 to robot_dof

# Create sequences for tau, tau_d, tau_ext, q, q_d, dq, dq_d, e, de, and etau
tau = generate_sequence('tau_J', indices)
tau_d = generate_sequence('tau_J_d', indices)
tau_ext = generate_sequence('tau_ext', indices)

q = generate_sequence('q', indices)
q_d = generate_sequence('q_d', indices)

dq = generate_sequence('dq', indices)
dq_d = generate_sequence('dq_d', indices)

e = generate_sequence('e', indices)
de = generate_sequence('de', indices)
etau = generate_sequence('etau_J', indices)


In [7]:
def split_data(dataset, train_split_rate = 0.75):
    msk = np.random.rand(len(dataset)) < train_split_rate
    train = dataset.loc[msk, :]
    test = dataset.loc[~msk, :]
    return train, test

def make_sequence(df, selected_features,seq_num, deltaMin, gap):
    contact_indexs_ = df.loc[df.label.diff()>0.05,:].index
    contact_indexs2 = df.loc[df.label.diff()<-0.05,:].index
    contact_indexs = [idx for idx, idx2 in zip(contact_indexs_, contact_indexs2) if idx2 - idx >= seq_num]

    dataset_df = pd.DataFrame()
    for contact_index in contact_indexs:
        window_right_edge = contact_index
        end_point = contact_index + seq_num 
        for step in range(window_right_edge , end_point, gap):
            label = df.label[step-1]
            window = df[selected_features][step-seq_num:step]
            df_dummy=pd.DataFrame(np.insert(window.values.flatten(), 0, label).reshape(1,-1))
            dataset_df = pd.concat([dataset_df,df_dummy],ignore_index=True)
    return dataset_df

# training data

In [10]:
main_path = os.getcwd().replace('AIModels','')

#save_path = main_path+'/dataset/test_dataset_target_robot/'
#main_path = main_path + 'frankaRobot/DATA/dataset/target_robot/'

save_path = main_path+'dataset/e_edot/localization_gap_10/'
main_path = os.path.expanduser('~/myProjects/contact_localization_dataset/dataset/extracted_data/')

os.makedirs(save_path, exist_ok=True)

selected_features = e + de
seq_num = 28
gap = 10 # 25 ms

columns = range(seq_num*len(selected_features)+1)
train_split_rate = 0.75

dict_label = {'a': 7, 'b':6, 'c':5, 'd':4, 'e':3, 'f':2, 'g':1}


for deltaMin in [0]: #range(int(seq_num/2)):
    df_master_train = pd.DataFrame(columns=columns)
    df_master_test = pd.DataFrame(columns=columns)
    for i in os.listdir(main_path):
        if len(i.split('.'))==1:

            df_conf_train = pd.DataFrame(columns=columns)
            df_conf_test = pd.DataFrame(columns=columns)

            submain_path = main_path+i+'/'
            print(submain_path)
            for tag_name in os.listdir(submain_path):
                if len(tag_name.split('.'))==1:
                    file_path = submain_path+tag_name+'/labeled_data.csv'
                    df = pd.read_csv(file_path)
                    df.drop(columns='index', inplace=True)
                    df = make_sequence(df, selected_features,seq_num, deltaMin, gap)
                    #labeling data
                    df[0] = df[0]*dict_label[tag_name[0]]
                    #print(df.loc[:,0].min())
                    train, test = split_data(df, train_split_rate)
                    df_conf_train = df_conf_train.append(train, ignore_index=True)
                    df_conf_test = df_conf_test.append(test, ignore_index=True)

        df_master_train = df_master_train.append(df_conf_train, ignore_index=True)
        df_master_test = df_master_test.append(df_conf_test, ignore_index=True)

    df_master_train.to_pickle(save_path+str(deltaMin)+'dataset_train.pkl')
    df_master_test.to_pickle(save_path+str(deltaMin)+'dataset_test.pkl')
    print(deltaMin, df_master_test.shape)


/home/rzma/myProjects/contact_localization_dataset/dataset/extracted_data/configuration5/
/home/rzma/myProjects/contact_localization_dataset/dataset/extracted_data/configuration1/
/home/rzma/myProjects/contact_localization_dataset/dataset/extracted_data/configuration4/
/home/rzma/myProjects/contact_localization_dataset/dataset/extracted_data/configuration2/
/home/rzma/myProjects/contact_localization_dataset/dataset/extracted_data/configuration6/
/home/rzma/myProjects/contact_localization_dataset/dataset/extracted_data/in_motion/
/home/rzma/myProjects/contact_localization_dataset/dataset/extracted_data/configuration3/
/home/rzma/myProjects/contact_localization_dataset/dataset/extracted_data/configuration7/
/home/rzma/myProjects/contact_localization_dataset/dataset/extracted_data/configuration8/
0 (6415, 197)


In [10]:
main_path = os.getcwd().replace('AIModels','')

#save_path = main_path+'/dataset/test_dataset_target_robot/'
#main_path = main_path + 'frankaRobot/DATA/dataset/target_robot/'

save_path = main_path+'dataset/test_dataset_source_robot/'
main_path = main_path + 'frankaRobot/DATA/dataset/source_robot/'

os.makedirs(save_path, exist_ok=True)

selected_features = e
seq_num = 28
gap = 5 # 25 ms

columns = range(seq_num*len(selected_features)+1)

df_master = pd.DataFrame(columns=columns)
for i in os.listdir(main_path):
    if len(i.split('.'))==1:
        file_path = main_path+i+'/labeled_data.csv'
        df = pd.read_csv(file_path)
        df.drop(columns='index', inplace=True)
        df = make_sequence(df, selected_features,seq_num, gap)
        #labeling data
        if 'link5' in i:
            df[0] = df[0]*5
        elif 'link6' in i:
            df[0] = df[0]*6
        elif 'f' in i:
            df[0] = df[0]*2
        elif 'e' in i:
            df[0] = df[0]*3
        #print(df.loc[:,0].min())
        df_master = df_master.append(df, ignore_index=True)

df_master.to_pickle(save_path+'dataset_test.pkl')
print(df_master.shape)


(378, 197)


In [17]:
robot_dof = 6
main_path = os.getcwd().replace('AIModels','')


save_path = main_path+'/dataset/test_dataset_target_robot_ur5/'
main_path = main_path + '/urRobot/DATA/dataset/UR5/'
'''
save_path = main_path + '/dataset/test_dataset_target_robot_ur10/'
main_path = main_path + '/urRobot/DATA/dataset/UR10/'
'''
# Function to generate sequence names
def generate_sequence(base_name, indices):
    return [f'{base_name}{i}' for i in indices]

# Automate the sequence generation using the function
indices = range(robot_dof)  # Since the indices go from 0 to robot_dof
selected_features = generate_sequence('e', indices)

os.makedirs(save_path, exist_ok=True)
seq_num = 28
gap = 5 # 25 ms

columns = range(seq_num*len(selected_features)+1)

df_master = pd.DataFrame(columns=columns)
for i in os.listdir(main_path):
    if len(i.split('.'))==1:
        file_path = main_path+i+'/labeled_data.csv'
        df = pd.read_csv(file_path)
        df = make_sequence(df, selected_features,seq_num, gap)
        #labeling data
        if 'link5' in i:
            df[0] = df[0]*5
        elif 'link6' in i:
            df[0] = df[0]*6
        elif 'link3' in i:
            df[0] = df[0]*3
        elif 'link4' in i:
            df[0] = df[0]*4
        #print(df.loc[:,0].min())
        df_master = df_master.append(df, ignore_index=True)

df_master.to_pickle(save_path+'dataset_test.pkl')
print(df_master.shape)


(1950, 169)
